In [11]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from valhalla import Actor, get_config


def get_isochrone(query: dict, actor: Actor):
    result = {}
    isochrones = actor.isochrone(query)
    for contour_ix, isochrone in enumerate(isochrones["features"]):
        geom = isochrone["geometry"]["coordinates"]
        time = isochrone["properties"]["contour"]
        result[time] = Polygon(geom)
    return result


def build_walk_query(
    location: dict[str, float], times: list[int] = [5, 10, 15, 20]
) -> dict:
    return {
        "locations": [location],
        "costing": "pedestrian",
        "contours": [{"time": i} for i in times],
    }


def build_bicycle_query(location: dict[str, float], times: list[int] = [10, 15, 30]):
    return {
        "locations": [location],
        "costing": "bicycle",
        "contours": [{"time": i} for i in times],
    }


def build_car_query(location: dict[str, float], times: list[int] = [10, 15, 30]):
    return {
        "locations": [location],
        "costing": "auto",
        "contours": [{"time": i} for i in times],
    }


def determine_isochrones(
    stops: gpd.GeoDataFrame, actor: Actor, times: list[int] = [5, 10, 15]
) -> pd.DataFrame:
    isochones = []
    for row in stops.itertuples():
        w = get_isochrone(
            build_walk_query({"lon": row.stop_lon, "lat": row.stop_lat}, times=times),
            actor,
        )
        b = get_isochrone(
            build_bicycle_query(
                {"lon": row.stop_lon, "lat": row.stop_lat}, times=times
            ),
            actor,
        )
        for t in times:
            isochones.append([row.stop_id, w[t], "walk", t])
            isochones.append([row.stop_id, b[t], "bicycle", t])
    return pd.DataFrame.from_records(
        isochones, columns=["stop_id", "geometry", "costing", "range"]
    )


def initialize_valhalla(city: str) -> Actor:
    config = get_config(
        tile_extract=f"../data/valhalla/{city}/valhalla_tiles.tar",
        verbose=True,
    )

    config["service_limits"]["isochrone"]["max_contours"] = 10
    config["service_limits"]["isochrone"]["max_locations"] = 10_000
    config["service_limits"]["isochrone"]["max_distance"] = 100_000

    # instantiate Actor to load graph and call actions
    return Actor(config)

In [12]:
CITY = "rotterdam"

In [13]:
Path(f"../output/{CITY}").mkdir(parents=True, exist_ok=True)

actor = initialize_valhalla(CITY)

2025/03/25 11:31:21.006540 [INFO] Tile extract successfully loaded with tile count: 59
2025/03/25 11:31:21.006640 [WARN] (stat): /data/valhalla/traffic.tar No such file or directory
2025/03/25 11:31:21.006651 [WARN] Traffic tile extract could not be loaded


In [14]:
stops = pd.read_csv(f"../data/stops/{CITY}/stops_with_centrality.csv", engine="pyarrow")
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
stops.head(3)

,Node,Eigenvector Centrality,Degree Centrality,Closeness Centrality,Betweenness Centrality,stop_id,clust,stop_lat,stop_lon,stop_name,geometry
0,1.0,0.000072,0.009302,0.095768,0.006561,2521113,1,51.914311,4.458813,"Rotterdam, Claes de Vrieselaan / Mathenesserlaan",POINT (4.45881 51.91431)
1,1.0,0.000072,0.009302,0.095768,0.006561,2522415,1,51.914240,4.458960,"Rotterdam, Claes de Vrieselaan / Mathenesserlaan",POINT (4.45896 51.91424)
2,1.0,0.000072,0.009302,0.095768,0.006561,2807351,1,51.914311,4.458813,"Rotterdam, Claes de Vrieselaan / Mathenesserlaan",POINT (4.45881 51.91431)


In [15]:
isochones = determine_isochrones(stops, actor)
isochones.to_csv(f"../output/{CITY}/isochrones.csv", index=False)

isochrones_gdf = gpd.GeoDataFrame(isochones, geometry="geometry", crs=4326)
isochrones_gdf.to_file(f"../output/{CITY}/isochrones.geojson")

In [ ]:
# isochones = pd.read_csv(f"../output/{CITY}/isochrones.csv")
# print(isochones.head(6).to_markdown(index=False))